## Assignment 4
***
*General hints:* <br>
* You may use another notebook to test different approaches and ideas. When complete and mature, turn your code snippets into the requested functions in this notebook for submission. 
* Make sure the function implementations are generic and can be applied to any dataset (not just the one provided).
* Add explanatory code comments in the code cells. Make sure that these comments improve our understanding of your implementation decisions.

-----
* Create a variable holding your student id, as shown below. 
* Simply replace the example (`01234567`) with your actual student id having a total of 8 digits. 
* Maintain the variable as a string, do NOT change its type in this notebook!
* *Note: If your student id has 7 digits, add a leading 0. The final student id MUST have 8 digits!*

In [1]:
mn = '12318768'

In [2]:
import pytest
import pandas as pd 
import numpy as np

## 0. Import

Implement a function `tidy` which imports the data set assigned and provided to you as a CSV file into a `pandas` dataframe. Access the data set and establish whether your data set is tidy. If not, clean the data set before continuing with Step 1. Mind all rules of tidying data sets in this step. Make sure you comply to the following statements:
* If there is an index column (row numbers) in your tidied dataset, keep it.
* The following columns, once identified, correspond to variables 1:1 (no need for transformations):
  * `full_name`
  * `automotive`
  * `color`
  * `job`
  * `address`
  * `coordinates`
  * `km_per_litre`
* The tidied dataset should have a total of 9 columns (not including the index), the first column should be `full_name` and the last one `km_per_litre`.
* Mind the intended content of each attribute (e.g. `full_name` should contain the full name of a person, no need to change that)
* If tidy or done, have the function `tidy` return the ready data set as a dataframe.

Note that `tidy` must take a single parameter that holds your student id (`mn`) as one part of the basename (according to the CoC) of the CSV file (i.e., the CoC file name without file extension). Change the name of the data file so that it matches this requirement and the CoC and make sure you submit your final ZIP following the Code of Conduct (CoC) requirements. Especially, make sure you put your data file in a folder called `data/` when submitting.

In [3]:
def tidy(x): #this function will tidy the data
    file_path = f"data/{x}.csv" #first we fetch the file path
    my_data_frame = pd.read_csv(file_path, header=None) #then we read the file

    my_data_frame_indexed = my_data_frame.set_index(0) #we set the first column as the index
    df = my_data_frame_indexed.T # and after looking at the data it seems to be transposed, so we transpose it again 

    first_col_name = df.columns[0] #now we can see that the first column is the full name of the person
    if pd.isna(first_col_name): #if the first column name is missing, we drop it
        df = df.drop(columns=[first_col_name])

    combined_column = df["date_time/full_company_name"] #now lets start splitting the name and date column

    date_time_part = combined_column.str[:26] # after looking at the data we can see that the date and time are the first 26 characters, so we can split it after 26 characters
    company_name_part = combined_column.str[26:] #and the rest is the company name

    df["date_time"] = date_time_part.str.strip() #now we can create the new columns, that are stripped
    df["company_name"] = company_name_part.str.strip() #and once again

    df = df.drop(columns=["date_time/full_company_name"]) #and now we remove the original column

    desired_column_order = [ #lets reorder our headers, so they match the asignment
        "full_name",
        "automotive",
        "color",
        "job",
        "address",
        "coordinates",
        "date_time",      # New column
        "company_name",   # New column
        "km_per_litre"    # and lets Ensure this is last
    ]
    df = df[desired_column_order] # and now we reorder the columns

    # and finally we return the dataframe
    return df

In [4]:
assert type(tidy(mn)) == pd.core.frame.DataFrame, "T0.1"
assert len((tidy(mn)).columns) == 9, "T0.2"
assert list((tidy(mn)).columns)[0] == "full_name", "T0.3"
assert list((tidy(mn)).columns)[len((tidy(mn)).columns)-1] == "km_per_litre", "T0.4"

In [5]:
# Edit this cell or remove it, and you shall perish, meow! 😼⚡️

-------
## 1. Missing values

### 1.1 Code part
Implement a function called `missing_values` which takes as an input a dataframe and check if there are any missing values in the dataset. Record the row positions (*not* the row labels!) of the observations containing missing values as a list of numbers and make sure that the function returns the recorded list in the end, sorted in ascending order. If there are no missing values, `missing_values` should return an empty list.

**NOTE:** You shall find out how missing values are encoded in your datasest and which missing values occur in your dataset, you will ***need manual inspection*** by applying Python helpers. For instance, missing values could be encoded as: `"nan"`,`"(+/-)inf"` but also other values or empty fields or fields containing only white spaces are conceivable to encode missing values in your dataset. Do *not* rely on built-in Python or pandas functions alone!

Important: Mind the difference between row positions and row labels. `.index` of a dataframe returns row labels. `.iloc` takes row positions.

In [6]:
import pandas as pd
# Note: numpy import removed as pd.isna handles np.nan

# Finds rows in DataFrame 'x' containing missing values.
# Checks for NaN/None, empty/whitespace strings, and common missing-value words.
def missing_values(x):
    # Store indices of rows with missing data
    missing_row_indices = []

    # Define common string representations of missing data (lowercase)
    missing_value_strings = ["n/a", "na", "null", "none", "nan", "inf", "-inf", "+inf", ""]

    # Iterate through each row index
    for i in range(len(x)):
        # Get the current row by position
        row = x.iloc[i]

        # Check each item in the row
        for item in row:
            # Check 1: Standard pandas check for NaN/None
            if pd.isna(item):
                missing_row_indices.append(i)
                break # Move to next row once a missing value is found

            # Check 2: Check for designated missing strings (or empty after stripping)
            try:
                # Convert to string, strip whitespace, convert to lowercase
                item_str_lower = str(item).strip().lower()
                if item_str_lower in missing_value_strings:
                    missing_row_indices.append(i)
                    break # Move to next row
            except Exception:
                # Handle potential errors during string conversion (optional, but safer)
                # If conversion fails, might indicate an unusual data type we treat as non-missing here
                pass

    # Sort the indices for consistent output (and remove potential duplicates if break was removed)
    # Using set() then list() is a common way to get unique sorted values
    final_sorted_list = sorted(list(set(missing_row_indices)))

    # Return the list of row indices with missing values
    return final_sorted_list

# --- Example Usage ---
# Assume 'tidy' function and 'mn' variable are defined already
# df = tidy(mn)
#
# print("Running missing values check...")
# missing_indices = missing_values(df)
#
# # Inspect the results outside the function:
# print(f"\nTotal rows checked: {len(df)}")
# print(f"Indices of rows with missing values: {missing_indices}")
# print(f"Count of rows with missing values: {len(missing_indices)}")
#
# # Optional: View the actual rows with missing data
# # if missing_indices:
# #     print("\nRows with missing data:")
# #     print(df.iloc[missing_indices])


In [7]:
assert type(missing_values(tidy(mn))) == list, "T1.1"
assert all(isinstance(i, int) for i in missing_values(tidy(mn))), "T1.2"

In [8]:
# Edit this cell or remove it, and you shall perish, meow! 😼⚡️

### 1.2. Analytical part

* Does the dataset contain missing values?
* Explain your manual-inspection procedure and the Python helpers used!
* If no, explain how you proved that this is actually the case. 
* If yes, describe the discovered missing values. What could be an explanation for their missingness?

Write your answer in the markdown cell bellow. Do NOT delete or replace the answer cell with another one!


YOUR ANSWER HERE

------
## 2. Handling missing values
### 2.1. Code part
Apply a (simple) function called *handling_missing_values* for handling missing values using an adequate single-imputation technique (or, one of the alternatives to single imputation) of your choice per type of missing values. Make use of the techniques learned in Unit 4. The function should take as an input a dataframe and return the updated dataframe. Mind the following:
- The objective is to apply single imputation on these synthetic data. Do not make up a background story (at this point)!
- Do NOT simply drop the missing values. This is not an option.
- The imputation technique must be adequate for a given variable type (quantitative, qualitative).
- To establish whether a variable is quantitative or qualitative, it is *not* sufficient to only inspect on data types!

In [9]:
import pandas as pd
import numpy as np

# Assume 'tidy' and 'missing_values' functions are defined elsewhere.

def handling_missing_values(x):
    """
    Cleans and fills missing values in a DataFrame using specific strategies
    for different columns.

    Args:
        x (pd.DataFrame): The input DataFrame.

    Returns:
        pd.DataFrame: A new DataFrame with missing values handled.
    """
    df = x.copy() # Work on a copy to avoid changing the original

    # Define common ways missing data might be represented as text
    missing_markers = ["n/a", "na", "null", "none", "nan", "inf", "-inf", "+inf", "", "NaN"]
    # Convert all these markers to NumPy's standard NaN (Not a Number)
    df = df.replace(missing_markers, np.nan)

    # Define text values (lowercase) that might be mistaken for missing data,
    # used later to check if a calculated 'mode' is suitable for filling.
    problematic_mode_strings = {s.lower() for s in missing_markers if isinstance(s, str)}

    # Go through each column and apply a filling strategy
    for col in df.columns:
        if col == "km_per_litre":
            # For numerical 'km_per_litre', fill with the column's average (mean)
            df[col] = pd.to_numeric(df[col], errors='coerce') # Ensure it's numeric
            mean_val = df[col].mean()
            if pd.notna(mean_val): # Only fill if mean could be calculated
                df[col] = df[col].fillna(mean_val)
            # Optional: could add an else clause to fill with 0 if mean is NaN

        elif col == "coordinates":
            # For 'coordinates', use a specific placeholder
            df[col] = df[col].fillna("(0,0)")

        else:
            # For 'date_time' and all other columns, try filling with the mode (most frequent value).
            # If mode is problematic or doesn't exist, use a default.
            default_fill = "1970-01-01 00:00:00" if col == "date_time" else "Unknown"
            fill_value = default_fill # Start with the default

            mode_result = df[col].mode()
            # Check if a mode exists and if it's suitable
            if not mode_result.empty:
                potential_mode = mode_result[0]
                # Check if the mode itself is NaN or looks like one of our missing markers
                is_problematic = pd.isna(potential_mode) or \
                                 (isinstance(potential_mode, str) and \
                                  potential_mode.strip().lower() in problematic_mode_strings)

                if not is_problematic:
                    fill_value = potential_mode # Use the mode if it's valid

            # Fill any remaining NaNs in the column with the chosen value
            df[col] = df[col].fillna(fill_value)

    return df

# --- Example Usage & Assertions (Keep assertions for validation) ---
# Assume 'mn' points to your data source and 'tidy'/'missing_values' are ready.
# df_tidied = tidy(mn)
# df_handled = handling_missing_values(df_tidied)
# missing_after_handling = missing_values(df_handled)

# # These assertions help confirm the function worked as expected.
# assert len(missing_after_handling) == 0, f"T2.1: Found {len(missing_after_handling)} missing values after handling!"
# assert df_handled.shape == df_tidied.shape, "T2.2: DataFrame shape changed during handling!"

# print("Handling missing values complete. Assertions passed!") # Optional confirmation

In [10]:
assert len(missing_values(handling_missing_values(tidy(mn)))) == 0, "T2.1"
assert handling_missing_values(tidy(mn)).shape == tidy(mn).shape, "T2.2"

### 2.2. Analytical part
Discuss the implications. Answer the following:

- How would you qualify the data-generating processes leading to different types of missing values, provided that the data was not synthetic?
- What are the benefits and disadvantages of the chosen single-imputation technique?
- How would you apply a multiple-imputation technique to one type of missing values, if applicable at all?
- We asked you to test for/treat as missing values by checking certain field values, as well as empty fields or fields containing the numeric value 0... what are potential problems of this heuristics?

Write your answer in the markdown cell bellow. Do NOT delete or replace the answer cell with another one!

YOUR ANSWER HERE

-----
## 3. Detecting duplicate entries
Implement a function called `duplicates` that takes as an input a (tidy) dataframe `x` and a list of column labels (`VARIABLES`). Assume that `duplicates` receives a dataframe as returned from your Step 0 implementation of `tidy`. It then checks whether there are any duplicates in the dataset. Record the row positions of the second and any later observations being duplicates and have `duplicates` return the list of rows positions, sorted in asending order, in the end. An empty list indicates the absence of duplicated observations.

Important:
* The first observation that belongs to the detected duplicates is *not* considered a duplicate!
* Mind the difference between row positions and row labels. `.index` of a dataframe returns row labels. `.iloc` takes row positions.

In [11]:
df = tidy(mn)
VARIABLES = df.columns.tolist(); # Change value assignment!
  # or manually define like ["full_name", "automotive", "color"]

# ✅ Function to find duplicate entries
def duplicates(x, vars):
    if not vars or not isinstance(vars, list) or not all(v in x.columns.tolist() for v in vars):
        return "Name variables defining potential duplicates!"
    
    # Find duplicated rows (excluding the first occurrence)
    duplicate_mask = x.duplicated(subset=vars, keep='first')

    # Convert row labels to row **positions**
    duplicate_positions = [x.index.get_loc(i) for i in x[duplicate_mask].index]

    return sorted(duplicate_positions)

# Run the function
df = tidy(mn)
duplicate_rows = duplicates(df, VARIABLES)

# Print diagnostics
print("🧠 Checking for duplicates using variables:", VARIABLES)
print("🔁 Duplicate row positions:", duplicate_rows)
print("🔢 Number of duplicates:", len(duplicate_rows))

# Optionally show duplicate row content
if duplicate_rows:
    print("\n📋 Duplicate row contents:")
    display(df.iloc[duplicate_rows])
else:
    print("✅ No duplicates found!")
    
    


🧠 Checking for duplicates using variables: ['full_name', 'automotive', 'color', 'job', 'address', 'coordinates', 'date_time', 'company_name', 'km_per_litre']
🔁 Duplicate row positions: [754]
🔢 Number of duplicates: 1

📋 Duplicate row contents:


,full_name,automotive,color,job,address,coordinates,date_time,company_name,km_per_litre
755,Chad Chase,2V TU960,SkyBlue,Environmental education officer,South Kimberlyberg,"(Decimal('66.381533'), Decimal('-6.316750'))",2005-06-30 06:46:05.814620,Pena and Sons,23


In [12]:
df = tidy(mn);
assert len(VARIABLES) > 0 and all([v in df.columns.tolist() for v in VARIABLES]), "T3.1"
assert duplicates(df, [list]) == "Name variables defining potential duplicates!", "T3.2"
assert duplicates(df, None) == "Name variables defining potential duplicates!", "T3.3"
assert type(duplicates(df, vars = df.columns.tolist())) == list, "T3.4"
assert all(isinstance(i, int) for i in duplicates(df, df.columns.tolist())), "T3.5"

In [13]:
# Edit this cell or remove it, and you shall perish, meow! 😼⚡️


-----
## 4. Detecting outliers
### 4.1. Code part
Implement a function called `detecting_outliers` to detect outliers in one selected quantitative variable. Pick a suitable variable from the tidied dataset based on your characterisation and apply one suitable outlier-detection technique as covered in Unit 4. Justify your choice of this technique in the analytical part. Again, the function is assumed to receive a tidied data set from Step 0. The function returns the row positions (*not* row labels!) of the rows containing outliers on the selected variable, sorted in ascending order.

In [14]:
import pandas as pd
import numpy as np # Often used alongside pandas

def detecting_outliers(x):
    """
    Finds outliers in the 'km_per_litre' column of a DataFrame using the IQR method.

    Args:
        x (pd.DataFrame): The input DataFrame.

    Returns:
        list: A sorted list of index labels for the outlier rows.
    """
    col_name = "km_per_litre"
    df = x.copy() # Work on a copy

    # Ensure the column is numeric, converting errors to NaN
    df[col_name] = pd.to_numeric(df[col_name], errors='coerce')

    # Calculate IQR bounds
    Q1 = df[col_name].quantile(0.25)
    Q3 = df[col_name].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Identify outliers
    is_outlier = (df[col_name] < lower_bound) | (df[col_name] > upper_bound)
    outlier_labels = df.index[is_outlier].tolist()

    return sorted(outlier_labels)

In [15]:
df = tidy(mn);
assert type(detecting_outliers(df)) == list, "T4.1"
assert all(isinstance(i, int) for i in detecting_outliers(df)), "T4.2"
assert len(detecting_outliers(df)) > 0 and len(detecting_outliers(df)) < .05*df.shape[0]


### 4.2. Analytical part
Discuss the implications. 

- What is the chosen outlier-detection technique? Explain it using your own words in 3-4 sentences.
- Describe the outliers detected: How many? How do they relate to the typical, non-outlier values in the remaining dataset?
- What could be one reason these outliers appear in the dataset? How would you treat them further?

Write your answer in the markdown cell below. Do NOT delete or replace the answer cell with another one!

YOUR ANSWER HERE